# Q1

In [2]:
import chess
import reconchess
from reconchess.utilities import without_opponent_pieces, is_illegal_castle

def generate_all_possible_moves(fen):
    board = chess.Board(fen)
    possible_moves = set()

    # 1. Add all pseudolegal moves
    for move in board.pseudo_legal_moves:
        possible_moves.add(move.uci())

    # 2. Add the null move
    possible_moves.add('0000')

    # 3. Add special castling moves permitted in RBC
    castle_board = without_opponent_pieces(board)
    for move in castle_board.generate_castling_moves():
        if not is_illegal_castle(board, move):
            possible_moves.add(move.uci())

    # 4. Output sorted list of moves
    for move in sorted(possible_moves):
        print(move)

# Sample Input
fen_input = input("Enter FEN string: ").strip()
generate_all_possible_moves(fen_input)


Enter FEN string: 8/8/8/8/7q/p2p1p1k/P2P1P2/Rn2K2R w KQ - 23 30
0000
a1b1
e1d1
e1e2
e1f1
e1g1
h1f1
h1g1
h1h2
h1h3


# Q2

In [4]:

def generate_next_fens(fen):
    board = chess.Board(fen)
    next_fens = set()

    # 1. Pseudolegal moves
    for move in board.pseudo_legal_moves:
        new_board = board.copy()
        new_board.push(move)
        next_fens.add(new_board.fen())

    # 2. Null move (0000)
    new_board = board.copy()
    new_board.push(chess.Move.null())
    next_fens.add(new_board.fen())

    # 3. Special castling in RBC
    castle_board = without_opponent_pieces(board)
    for move in castle_board.generate_castling_moves():
        if not is_illegal_castle(board, move):
            new_board = board.copy()
            new_board.push(move)
            next_fens.add(new_board.fen())

    # Output sorted FENs
    for f in sorted(next_fens):
        print(f)

# Sample input
fen_input = input("Enter FEN string: ").strip()
generate_next_fens(fen_input)


Enter FEN string: k7/p2p1p2/P2P1P2/8/8/8/8/7K b - - 23 30
1k6/p2p1p2/P2P1P2/8/8/8/8/7K w - - 24 31
8/pk1p1p2/P2P1P2/8/8/8/8/7K w - - 24 31
k7/p2p1p2/P2P1P2/8/8/8/8/7K w - - 24 31


# Q3

In [6]:
import chess
from reconchess.utilities import without_opponent_pieces, is_illegal_castle

def generate_capture_resulting_fens(fen, capture_square_str):
    board = chess.Board(fen)
    target_square = chess.SQUARE_NAMES.index(capture_square_str)
    next_fens = set()

    # 1. Check all pseudolegal moves that end on the capture square
    for move in board.pseudo_legal_moves:
        if move.to_square == target_square and board.is_capture(move):
            new_board = board.copy()
            new_board.push(move)
            next_fens.add(new_board.fen())

    # 2. Special RBC castling (very rare, but handle it safely)
    castle_board = without_opponent_pieces(board)
    for move in castle_board.generate_castling_moves():
        if not is_illegal_castle(board, move):
            if move.to_square == target_square and board.is_capture(move):
                new_board = board.copy()
                new_board.push(move)
                next_fens.add(new_board.fen())

    # 3. Return sorted list
    for f in sorted(next_fens):
        print(f)

# Sample Input
fen_input = input("Enter FEN string: ").strip()
capture_square_input = input("Enter capture square (e.g., d6): ").strip().lower()
generate_capture_resulting_fens(fen_input, capture_square_input)


Enter FEN string: r1bqk2r/pppp1ppp/2n2n2/4p3/1b2P3/1P3N2/PBPP1PPP/RN1QKB1R w KQkq - 0 5
Enter capture square (e.g., d6): e5
r1bqk2r/pppp1ppp/2n2n2/4B3/1b2P3/1P3N2/P1PP1PPP/RN1QKB1R b KQkq - 0 5
r1bqk2r/pppp1ppp/2n2n2/4N3/1b2P3/1P6/PBPP1PPP/RN1QKB1R b KQkq - 0 5


# Q4

In [8]:
import chess

def parse_window(window_str):
    """
    Parses the window description into a dictionary:
    {'c8': '?', 'd8': '?', ..., 'd7': 'n'}
    """
    window = {}
    entries = window_str.strip().split(';')
    for entry in entries:
        if entry:
            square, piece = entry.split(':')
            window[square] = piece
    return window

def fen_matches_window(fen, window):
    """
    Checks if the given FEN matches the window observation.
    '?' means the square must be empty.
    Any other piece letter must match exactly.
    """
    board = chess.Board(fen)
    for square_str, expected_piece in window.items():
        square = chess.parse_square(square_str)
        piece = board.piece_at(square)

        if expected_piece == '?':
            if piece is not None:
                return False  # Expected empty square
        else:
            if piece is None or piece.symbol() != expected_piece:
                return False  # Piece mismatch
    return True

def filter_fens_by_window(fens, window_str):
    window = parse_window(window_str)
    consistent_fens = [fen for fen in fens if fen_matches_window(fen, window)]
    for fen in sorted(consistent_fens):
        print(fen)

N = int(input())
fens = [input().strip() for _ in range(N)]
window_description = input().strip()

filter_fens_by_window(fens, window_description)


3
1k6/1ppn4/8/8/8/1P1P4/PN3P2/2K5 w - - 0 32
1k6/1ppnP3/8/8/8/1P1P4/PN3P2/2K5 w - - 0 32
1k6/1ppn1p2/8/8/8/1P1P4/PN3P2/2K5 w - - 0 32
c8:?;d8:?;e8:?;c7:p;d7:n;e7:?;c6:?;d6:?;e6:?
1k6/1ppn1p2/8/8/8/1P1P4/PN3P2/2K5 w - - 0 32
1k6/1ppn4/8/8/8/1P1P4/PN3P2/2K5 w - - 0 32
1k6/1ppnP3/8/8/8/1P1P4/PN3P2/2K5 w - - 0 32
